# SQL-to-Text Model Evaluation

This notebook evaluates pre-trained models for SQL-to-text generation without any training.

## Features
- Evaluate any HuggingFace model (seq2seq or causal LM)
- Auto-detects model architecture
- Multiple metrics: BLEU, ROUGE, CHRF, LaBSE similarity
- Test on custom SQL queries
- Save evaluation results to CSV

In [ ]:
# Install dependencies
!pip install -q torch transformers datasets sacremoses sentence-transformers

## Configuration

Set the model and evaluation parameters:

In [ ]:
MODEL_NAME = "cointegrated/rut5-base"
DATA_DIR = "."
OUTPUT_FILE = "evaluation_results.json"
NUM_SAMPLES = 50
MAX_NEW_TOKENS = 100
TEMPERATURE = 0.7

print(f"Model: {MODEL_NAME}")
print(f"Samples to evaluate: {NUM_SAMPLES if NUM_SAMPLES else 'All'}")

In [ ]:
from google.colab import files
print("Please upload pauq_dev.json")
uploaded = files.upload()

## Data Loading Functions

In [ ]:
import json
import os
from typing import List, Dict

def load_pauq_data(data_dir: str, split: str = "dev") -> List[Dict]:
    filename = f"pauq_{split}.json"
    filepath = os.path.join(data_dir, filename)
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Data file not found: {filepath}")
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"Loaded {len(data)} examples from {filename}")
    return data

In [ ]:
def get_eval_subset(data: List[Dict], num_samples: int = None) -> List[Dict]:
    if num_samples:
        return data[:num_samples]
    return data

In [ ]:
dev_data = load_pauq_data(DATA_DIR, "dev")
eval_data = get_eval_subset(dev_data, NUM_SAMPLES)
print(f"\nEvaluating on {len(eval_data)} samples")

## Model Loading Functions

In [ ]:
import torch

def load_model_and_tokenizer(model_name: str):
    from transformers import AutoTokenizer
    print(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    try:
        from transformers import AutoModelForSeq2SeqLM
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        is_seq2seq = True
        print("Detected: Seq2Seq model")
    except (OSError, ValueError, KeyError):
        from transformers import AutoModelForCausalLM
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        is_seq2seq = False
        print("Detected: Causal LM")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer, is_seq2seq

In [ ]:
model, tokenizer, is_seq2seq = load_model_and_tokenizer(MODEL_NAME)
print(f"Model loaded on: {model.device}")
print(f"Model parameters: {model.num_parameters():,}")

## Generation Functions

In [ ]:
def generate_question(model, tokenizer, sql_query: str, is_seq2seq: bool,
                      max_new_tokens: int = 100, temperature: float = 0.7):
    if is_seq2seq:
        prompt = f"SQL: {sql_query}"
    else:
        prompt = f"SQL: {sql_query}\nQuestion:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )
    if is_seq2seq:
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    else:
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        result = full_output.split("Question:")[-1].strip()
    return result

## Quick Evaluation

Test the model on a few samples:

In [ ]:
print("\nSample predictions:")
print("=" * 80)
for i, item in enumerate(eval_data[:5]):
    sql_query = item.get("query", {}).get("en", "")
    actual_question = item.get("question", {}).get("en", "")
    predicted_question = generate_question(
        model, tokenizer, sql_query, is_seq2seq, MAX_NEW_TOKENS, TEMPERATURE
    )
    print(f"\n--- Sample {i+1} ---")
    print(f"SQL: {sql_query}")
    print(f"Expected: {actual_question}")
    print(f"Predicted: {predicted_question}")
    print("-" * 80)

## Evaluation Functions

Functions to compute metrics:

In [ ]:
import numpy as np

def compute_metrics(references, hypotheses):
    from sacrebleu.metrics import BLEU, CHRF
    from rouge_score import rouge_scorer
    from sentence_transformers import SentenceTransformer, util

    print("Computing BLEU...")
    bleu_metric = BLEU()
    bleu_result = bleu_metric.corpus_score(hypotheses, [references])

    print("Computing ROUGE...")
    rouge_scorer_instance = rouge_scorer.RougeScorer(
        ['rouge1', 'rouge2', 'rougeL'], use_stemmer=True
    )
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
    for ref, hyp in zip(references, hypotheses):
        scores = rouge_scorer_instance.score(ref, hyp)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    print("Computing CHRF...")
    chrf_metric = CHRF()
    chrf_result = chrf_metric.corpus_score(hypotheses, [references])

    print("Computing LaBSE similarity...")
    labse_model = SentenceTransformer('sentence-transformers/LaBSE')
    ref_embeddings = labse_model.encode(references, convert_to_tensor=True)
    hyp_embeddings = labse_model.encode(hypotheses, convert_to_tensor=True)
    similarities = util.cos_sim(hyp_embeddings, ref_embeddings)
    similarity_scores = torch.diagonal(similarities).cpu().numpy()

    return {
        'BLEU-1': bleu_result.precisions[0],
        'BLEU-2': bleu_result.precisions[1],
        'BLEU-3': bleu_result.precisions[2],
        'BLEU-4': bleu_result.precisions[3],
        'ROUGE-1': np.mean(rouge1_scores) * 100,
        'ROUGE-2': np.mean(rouge2_scores) * 100,
        'ROUGE-L': np.mean(rougeL_scores) * 100,
        'CHRF': chrf_result.score,
        'LaBSE-Similarity': np.mean(similarity_scores) * 100,
    }

In [ ]:
def run_evaluation(model, tokenizer, eval_data, is_seq2seq,
                   max_new_tokens, temperature, output_file=None):
    references, hypotheses, predictions = [], [], []
    print(f"\nEvaluating {len(eval_data)} samples...")

    for item in eval_data:
        sql_query = item.get("query", {}).get("en", "")
        actual_question = item.get("question", {}).get("en", "")
        predicted = generate_question(
            model, tokenizer, sql_query, is_seq2seq, max_new_tokens, temperature
        )
        references.append(actual_question)
        hypotheses.append(predicted)
        predictions.append({
            'id': item.get('id', ''),
            'sql': sql_query,
            'expected': actual_question,
            'predicted': predicted,
        })

    print("Computing metrics...")
    metrics = compute_metrics(references, hypotheses)

    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(predictions, f, indent=2, ensure_ascii=False)
        print(f"\nPredictions saved to {output_file}")

    return metrics

## Run Full Evaluation

In [ ]:
metrics = run_evaluation(
    model, tokenizer, eval_data, is_seq2seq,
    MAX_NEW_TOKENS, TEMPERATURE, OUTPUT_FILE
)
print("\n" + "=" * 80)
print("EVALUATION RESULTS")
print("=" * 80)
print(f"\nModel: {MODEL_NAME}")
print(f"Dataset: {len(eval_data)} samples")
print("\n--- Metrics ---")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")
print("=" * 80)

## Custom SQL Test

Test with your own SQL queries:

In [ ]:
test_sqls = [
    "SELECT name FROM users WHERE age > 25;",
    "SELECT COUNT(*) FROM orders WHERE status = 'completed';",
    "SELECT product, SUM(quantity) FROM sales GROUP BY product ORDER BY SUM(quantity) DESC;",
]
print("\nCustom SQL Tests:")
print("=" * 80)
for i, test_sql in enumerate(test_sqls):
    predicted = generate_question(
        model, tokenizer, test_sql, is_seq2seq, MAX_NEW_TOKENS, TEMPERATURE
    )
    print(f"\n--- Test {i+1} ---")
    print(f"SQL: {test_sql}")
    print(f"Generated Question: {predicted}")

## Compare Multiple Models

Define a list of models to compare:

In [ ]:
MODELS_TO_COMPARE = [
    "cointegrated/rut5-base",
    "google/flan-t5-base",
    "Qwen/Qwen2.5-0.5B-Instruct",
]
COMPARE_NUM_SAMPLES = 20
compare_data = get_eval_subset(dev_data, COMPARE_NUM_SAMPLES)
print(f"Comparing {len(MODELS_TO_COMPARE)} models on {len(compare_data)} samples...")
all_results = []

for model_name in MODELS_TO_COMPARE:
    print(f"\n{'=' * 80}")
    print(f"Evaluating: {model_name}")
    try:
        compare_model, compare_tokenizer, compare_is_seq2seq = load_model_and_tokenizer(model_name)
        compare_metrics = run_evaluation(
            compare_model, compare_tokenizer, compare_data, compare_is_seq2seq,
            MAX_NEW_TOKENS, TEMPERATURE, None
        )
        result = {'model': model_name}
        result.update(compare_metrics)
        all_results.append(result)
        del compare_model, compare_tokenizer
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"Error evaluating {model_name}: {e}")
        continue

In [ ]:
import pandas as pd
df_results = pd.DataFrame(all_results)
print("\n" + "=" * 80)
print("MODEL COMPARISON RESULTS")
print("=" * 80)
print(df_results.to_string(index=False))
df_results.to_csv('model_comparison.csv', index=False)
print("\nResults saved to model_comparison.csv")

## Download Results (Optional)

Download the evaluation results:

In [ ]:
print("\nStarting download...")
files.download(OUTPUT_FILE)
if os.path.exists('model_comparison.csv'):
    files.download('model_comparison.csv')